In [1]:
#!pip install keras-swa

In [2]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import StratifiedKFold, train_test_split
import os
import matplotlib.pyplot as plt

In [3]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score

Using TensorFlow backend.


In [4]:
from keras.layers import Dense, Input, Dropout, Activation, BatchNormalization, GaussianNoise
from keras.models import Model, load_model
from keras import optimizers, callbacks
from keras.optimizers import Adam

from keras.callbacks import ModelCheckpoint,Callback, EarlyStopping
from keras import backend as K
from swa.keras import SWA # swa


# 데이터 불러오기

In [5]:
train = pd.read_csv('./data/train.csv', index_col=0)
test = pd.read_csv('./data/test.csv', index_col=0)
sample_submission = pd.read_csv('./data/sample_submission.csv', index_col=0)

# Train 데이터의 타입을 Sample_submission에 대응하는 가변수 형태로 변환
column_number = {}
for i, column in enumerate(sample_submission.columns):
    column_number[column] = i
    
def to_number(x, dic):
    return dic[x]

train['type_num'] = train['type'].apply(lambda x : to_number(x, column_number))

# 모델에 적용할 데이터 셋 준비 
x = train.drop(columns=['type', 'type_num'], axis=1)
y = train['type_num']

x = x.drop(columns=['fiberID'], axis=1)
test_x = test.drop(columns=['fiberID'],axis=1)

x_name=x.columns
col_name=x_name

In [6]:
x=np.array(x)
y=np.array(y)
test_x=np.array(test_x)

In [7]:
from sklearn.model_selection import train_test_split, KFold, RandomizedSearchCV
x_train, x_test, y_train, y_test = train_test_split(x,y, stratify=y,\
                                                    train_size=0.9, shuffle=True ,random_state=66)

In [8]:
from keras.layers import Activation
from keras.utils.generic_utils import get_custom_objects
import tensorflow as tf

class Gelu(Activation):
    def __init__(self, activation, **kwargs):
        super(Gelu, self).__init__(activation, **kwargs)
        self.__name__='gelu'
        
def gelu(x):
    return 0.5 * x * (1 + tf.tanh(np.sqrt(2 / np.pi) * (x + 0.044715 * np.power(x, 3))))

get_custom_objects().update({'gelu': Gelu(gelu)})

# Scaler

In [9]:
from sklearn.preprocessing import StandardScaler, RobustScaler

# scaler1=RobustScaler()
scaler2=StandardScaler()

scaler2.fit(x_train)
x_train=scaler2.transform(x_train)
x_test=scaler2.transform(x_test)

test_x =scaler2.transform(test_x)

# Variables

In [10]:
lr1 = 3.e-4 ; lr1_d = 3.e-5 
lr2 = 2.e-4 ; lr2_d = 3.e-5
lr3 = 1.e-4 ; lr3_d = 3.e-5

drop1 = 0.2
drop2 = 0.15
drop3 = 0.1

In [11]:
import datetime
start=datetime.datetime.now()
print(start)

2020-02-18 20:37:45.405491


# Deep Learning Model
- EarlyStop, CosineScheduler, CheckPoint

In [12]:
def model_1(epochs=1200, drop=0.2, lr=lr1, lr_d=0):

    inps= Input(shape = (20,))

    la = Dense(128)(inps)
    la = Activation(gelu)(la)
    la = Dropout(drop1)(la)
    
    la = Dense(128)(la)
    la = Activation(gelu)(la)
    la = Dropout(drop1)(la)
    
    la = Dense(128)(la)
    la = Activation(gelu)(la)
    la = Dropout(drop1)(la)
    
    la = Dense(128)(la)
    la = Activation(gelu)(la)
    la = Dropout(drop1)(la)
    
    la = Dense(128)(la)
    la = Activation(gelu)(la)
    la = Dropout(drop1)(la)
    
    la = Dense(128)(la)
    la = Activation(gelu)(la)
    la = Dropout(drop1)(la)
    
    
    outs=Dense(19,activation='softmax',name='output')(la)

    models = Model(inputs=inps, outputs=outs)
    models.summary()
    models.compile(loss="sparse_categorical_crossentropy", \
                   optimizer=Adam(lr=lr1,decay=lr1_d/epochs),metrics=['accuracy'])
    return models


In [13]:
def model_2(epochs=1200, drop=0.2, lr=lr2, lr_d=0):

    inps= Input(shape = (20,))

    la = Dense(128)(inps)
    la = Activation(gelu)(la)
    la = Dropout(drop2)(la)

    la = Dense(256)(la)
    la = Activation(gelu)(la)
    la = Dropout(drop2)(la)
    
    la = Dense(256)(la)
    la = Activation(gelu)(la)
    la = Dropout(drop2)(la)
    
    la = Dense(128)(la)
    la = Activation(gelu)(la)
    la = Dropout(drop2)(la)
        
    la = Dense(128)(la)
    la = Activation(gelu)(la)
    la = Dropout(drop2)(la)
    
    outs=Dense(19,activation='softmax',name='output')(la)

    models = Model(inputs=inps, outputs=outs)
    models.summary()
    models.compile(loss="sparse_categorical_crossentropy", \
                   optimizer=Adam(lr=lr2,decay=3.e-5/epochs),metrics=['accuracy'])
    
    return models

In [14]:
def model_3(epochs=1200, drop=0.2, lr=lr3, lr_d=0):

    inps= Input(shape = (20,))

    la = Dense(128)(inps)
    la = Activation(gelu)(la)
    la = Dropout(drop3)(la)
    
    la = Dense(128)(la)
    la = Activation(gelu)(la)
    la = Dropout(drop3)(la)
    
    la = Dense(128)(la)
    la = Activation(gelu)(la)
    la = Dropout(drop3)(la)
    
    la = Dense(128)(la)
    la = Activation(gelu)(la)
    la = Dropout(drop3)(la)
    
    outs=Dense(19,activation='softmax',name='output')(la)

    models = Model(inputs=inps, outputs=outs)
    models.summary()
    models.compile(loss="sparse_categorical_crossentropy", \
                   optimizer=rmsprop(lr=lr3,decay=3.e-5/epochs),metrics=['accuracy'])
    
    return models

In [27]:
sk_params = { 'validation_split': 0.2 }

model_1a = KerasClassifier(build_fn = model_1, epochs = 10, drop=drop1, verbose=2, batch_size=512,\
                        lr=lr1, lr_d=lr1_d, **sk_params)

model_1b = KerasClassifier(build_fn = model_1, epochs = 10, drop=drop1, verbose=0, batch_size=512,\
                        lr=lr1, lr_d=lr1_d, **sk_params)

model_1c = KerasClassifier(build_fn = model_1, epochs = 10, drop=drop1, verbose=0, batch_size=512,\
                        lr=lr1, lr_d=lr1_d, **sk_params)

model_2a = KerasClassifier(build_fn = model_2, epochs = 10, drop=drop2, verbose=2, batch_size=512,\
                        lr=lr2, lr_d=lr2_d, **sk_params)

model_2b = KerasClassifier(build_fn = model_2, epochs = 10, drop=drop2, verbose=0, batch_size=512,\
                        lr=lr2, lr_d=lr2_d, **sk_params)

model_2c = KerasClassifier(build_fn = model_2, epochs = 10, drop=drop2, verbose=0, batch_size=512,\
                        lr=lr2, lr_d=lr2_d, **sk_params)

model_3a = KerasClassifier(build_fn = model_3, epochs = 10, drop=drop3, verbose=2, batch_size=512,\
                        lr=lr3, lr_d=lr3_d, **sk_params)

model_3b = KerasClassifier(build_fn = model_3, epochs = 10, drop=drop3, verbose=0, batch_size=512,\
                        lr=lr3, lr_d=lr3_d, **sk_params)

model_3c = KerasClassifier(build_fn = model_3, epochs = 10, drop=drop3, verbose=0, batch_size=512,\
                        lr=lr3, lr_d=lr3_d, **sk_params)


In [28]:
ensemble_clf = VotingClassifier(estimators = [('model1', model_1a), ('model2', model_1b), ('model3', model_1c),
                                              ('model4', model_2a), ('model5', model_2b), ('model6', model_2c),
                                              ('model7', model_3a), ('model8', model_3b), ('model9', model_3c),],
                                                 voting = 'soft')

In [29]:
ensemble_clf.fit(x_train, y_train)

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 20)                0         
_________________________________________________________________
dense_30 (Dense)             (None, 128)               2688      
_________________________________________________________________
activation_30 (Activation)   (None, 128)               0         
_________________________________________________________________
dropout_30 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_31 (Dense)             (None, 128)               16512     
_________________________________________________________________
activation_31 (Activation)   (None, 128)               0         
_________________________________________________________________
dropout_31 (Dropout)         (None, 128)               0   

KeyboardInterrupt: 

In [ ]:
print(ensemble_clf.score(x_test, y_test))

In [ ]:
end=datetime.datetime.now()
print("걸린 시간:", end-start)

pred_test=ensemble_clf.predict_proba(test_x)

In [ ]:
print(pred_test.shape)

# Model Predict

In [ ]:
# 제출 파일 생성
submission = pd.DataFrame(data=pred_test, columns=sample_submission.columns, index=sample_submission.index)
submission.to_csv('./data/submission_data_ensemble.csv', index=True)